In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import typing

In [8]:
anio = 2016

In [9]:
data = pd.read_csv(f'/home/jovyan/work/data/raw/{anio}.csv')

In [10]:
# This wrapper will apply any sklearn transform you pass into it to a group.
class SklearnWrapper:
    def __init__(self, transformation: typing.Callable):
        self.transformation = transformation

    def __call__(self, df):
        transformed = self.transformation.fit_transform(df.values)
        return pd.DataFrame(transformed, columns=df.columns, index=df.index)

In [11]:
grouped = data.groupby(['ORIGIN','FL_DATE']).agg({'DEP_DELAY': 'mean'})

df_rescaled = (
    grouped.groupby("ORIGIN")
    .apply(SklearnWrapper(StandardScaler()))
    .rename(columns = {'DEP_DELAY': 'SCALED'})
)
df_rescaled = (
    grouped.join(df_rescaled)
    .assign(OUTLIER=(abs(df_rescaled.SCALED)>=3).astype(int))
)

In [12]:
df_rescaled.head()

DEP_DELAY    SCALED  OUTLIER
ORIGIN FL_DATE                                 
ABE    2016-01-01  18.666667  0.341619        0
       2016-01-02  -0.666667 -0.412299        0
       2016-01-03  31.666667  0.848564        0
       2016-01-04   1.500000 -0.327808        0
       2016-01-05  -1.333333 -0.438296        0

In [13]:
df_rescaled.OUTLIER.sum()

2148